In [ ]:
import sys
import os
import numpy as np
import scipy.signal as sp
import processing as pr
import matplotlib.pyplot as plt
import pandas as pd
from ruamel.yaml import YAML as ym
import datetime
import copy
import re
import scipy.fft
import pickle

# Use this for interactive graphs - nice for exploring
#%matplotlib widget

%matplotlib inline

import mplcursors

sys.path.append("../preprocessing")
from generate_chirp import generate_chirp

# Widgets are only needed if you want to use the interactive plot at the end
# Installation instructions: https://ipywidgets.readthedocs.io/en/latest/user_install.html
import ipywidgets as widgets

In [ ]:
# 8-28 Vatnajokull Day 1

# prefix = "../../drone/radar_data/20220828-vatnajokull-day1/20220828_055107" # Flight 1
# px4_logs = "../../px4_logs/2022-08-28/12_40_43/12_40_43_"

# prefix = "../../drone/radar_data/20220828-vatnajokull-day1/20220828_071157" # Flight 2
# px4_logs = "../../px4_logs/2022-08-28/14_03_05/14_03_05_"

# 9-1 Vatnajokull Day 2
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_085756"
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_090136"
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_090856"
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_091051"
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_091247"
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_091437"
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_091643"

# prefix = "../../drone/radar_data/20220901-vatnajokull-day2/20220901_095129" # Flight 1 -- possibly too aggressive PRF
# px4_logs = "../../px4_logs/ 2022-09-01/16_40_36/16_40_36_"

# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_101053" # 350 MHz center for anna
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_102013" # back to regular -- rect window now
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_102245"
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_102720"
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_102935"

# prefix = "../../drone/radar_data/20220901-vatnajokull-day2/20220901_105725" # Flight 2
# px4_logs = "../../px4_logs/2022-09-01/17_43_54/17_43_54_"

# Vatnajokull Day 3 -- Wind, stuck in cabin

# Vatnajokull Day 4

# prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_030152"
# prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_033000"

# prefix = "../../drone/radar_data/20220903-vatnajokull-day4/20220903_035332" # Flight 1 - V2 to V1 transect
# px4_logs = "../../px4_logs/2022-09-03/10_40_26/10_40_26_"

# prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_045320" # 350 mhz center, longer pulse
# prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_045646" # 3 db more tx power
# prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_045933" # another 3db
# prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_062538" # 2 more db
# prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_062823" # 150 us pulse
# prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_063113" # same thing, more pulses
# prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_064218" # final check

prefix = "../../drone/radar_data/20220903-vatnajokull-day4/20220903_070453" # Flight 2
px4_logs = "../../px4_logs/2022-09-03/13_49_57/13_49_57_"

#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_082804" # 20 us pulse length, faster prf
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_083111" # faster prf
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_083343" # tried blackman window
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_083526" # back to rect
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_083713" # back to 390 mhz
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_083842" # blackman win
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_084054" # 3 db more
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_084251"
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_084933"
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_085122"
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_085245"
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_085723"

# prefix = "../../drone/radar_data/20220903-vatnajokull-day4/20220903_091901" # Flight 3
# px4_logs = "../../px4_logs/2022-09-03/16_03_58/16_03_58_"

# Vatnajokull Day 5 -- SKF6

#prefix = "../../radar_data/20220904-vatnajokull-day5/20220904_032849"

# prefix = "../../drone/radar_data/20220904-vatnajokull-day5/20220904_041423" # Flight 1.5 (forgot to turn on radar for first one)
# px4_logs = "../../px4_logs/2022-09-04/11_04_34/11_04_34_"

# prefix = "../../drone/radar_data/20220904-vatnajokull-day5/20220904_054559" # Flight 2 (the long one)
# px4_logs = "../../px4_logs/2022-09-04/12_10_47/12_10_47_"

# Slakbreen 1

#prefix = "../../radar_data/20230313-slakbreen-day1/20230312_043805" # Spot 1 -- New wings, 2x extensions
#prefix = "../../radar_data/20230313-slakbreen-day1/20230312_045810" # Same thing, longer measurement
#prefix = "../../radar_data/20230313-slakbreen-day1/20230312_054339" # Spot 2 -- New wings, 1x extensions
#prefix = "../../radar_data/20230313-slakbreen-day1/20230312_054746" # Same thing, longer measurement
#prefix = "../../radar_data/20230313-slakbreen-day1/20230312_070350" # Spot 3 -- Old (OG) wings, 1x extensions
#prefix = "../../radar_data/20230313-slakbreen-day1/20230312_070743" # Same thing, longer measurement

## Slakbreen 2

# prefix = "../../radar_data/20230314-slakbreen-day2/20230314_070258" # Ground tests, tuning gain
# prefix = "../../radar_data/20230314-slakbreen-day2/20230314_070632"
# prefix = "../../radar_data/20230314-slakbreen-day2/20230314_070854"
# prefix = "../../radar_data/20230314-slakbreen-day2/20230314_071121"

# prefix = "../../radar_data/20230314-slakbreen-day2/20230314_073911" # Flight 1
# px4_logs = "../../px4_logs/2023-03-14/14_34_05/14_34_05_"

# prefix = "../../radar_data/20230314-slakbreen-day2/20230314_080735" # Flight 2
# px4_logs = "../../px4_logs/2023-03-14/14_59_01/14_59_01_"

# Slakbreen 3

#prefix = "../../radar_data/20230315-slakbreen-day3/20230315_061311"
#prefix = "../../radar_data/20230315-slakbreen-day3/20230315_061607"

# prefix = "../../radar_data/20230315-slakbreen-day3/20230315_064228" # Flight 1
# px4_logs = "../../px4_logs/2023-03-15/13_35_40/13_35_40_"

#prefix = "../../radar_data/20230315-slakbreen-day3/20230315_070941" # Flight 2
#px4_logs = "../../px4_logs/2023-03-15/14_03_44/14_03_44_"

# prefix = "../../radar_data/20230315-slakbreen-day3/20230315_074655" # Flight 3
# px4_logs = "../../px4_logs/2023-03-15/14_38_58/14_38_58_"


# Slakbreen 4

# prefix = "../../radar_data/20230316-slakbreen-day4/20230316_071250"
# px4_logs = "../../px4_logs/2023-03-16/13_58_18/13_58_18_"

# Slakbreen 5

# prefix = "../../radar_data/20230317-slakbreen-day5/20230317_033041" # flight 1 (2x liion, 2x wing ext)
# px4_logs = "../../px4_logs/2023-03-17/10_20_31/10_20_31_"

prefix = "../../radar_data/20230317-slakbreen-day5/20230317_042115" # flight 2 (2x liion, 1x wing ext)
px4_logs = "../../px4_logs/2023-03-17/10_58_17/10_58_17_"

#prefix = "../../radar_data/20230317-slakbreen-day5/20230317_061000" # flight 3 (2x liion, 1x wing ext)
#px4_logs = "../../px4_logs/2023-03-17/12_47_41/12_47_41_"


#px4_logs = None

In [ ]:
# %load data_bench
# px4_logs = None

In [ ]:
yaml_file = prefix + "_config.yaml"
bin_file = prefix + "_rx_samps.bin"
log_file = prefix + "_uhd_stdout.log"

# For PX4 logs: They must be converted to a set of CSV files with pyulog
# For example: `ulog2csv 17_39_45.ulg -o 17_39_45/`

# Thomas's system at 56 MHz sample rate - zero_sample_idx = 159
zero_sample_idx = 159

In [ ]:
# Initialize Constants
yaml = ym()
with open(yaml_file) as stream:
    config = yaml.load(stream)
    sample_rate = config["PLOT"]["sample_rate"]    # Hertz
    sig_speed = config["PLOT"]["sig_speed"] / np.sqrt(3.17) ## ICE
    
    rx_len_samples = int(config['CHIRP']['rx_duration'] * config['GENERATE']['sample_rate'])
    
rx_samps = bin_file

config_blackman_window = copy.deepcopy(config)
config_blackman_window['GENERATE']['window'] = 'blackman'

# Read and plot RX/TX
# This cell loads all of the data - it can take a while with a large file. You don't need to re-run this cell if you only change n_stack
_, tx_sig = generate_chirp(config)
_, tx_sig_blackman_window = generate_chirp(config_blackman_window)
pr.plotChirpVsTime(tx_sig, 'Transmitted Chirp', sample_rate)
print(f"len(tx_sig): {len(tx_sig)}")

t0 = datetime.datetime.now()

max_chunk_size_samples = int(2e8)
max_seconds_to_load = 60*100
load_start_seconds = 0

max_file_size_bytes = rx_len_samples*int(1/config['CHIRP']['pulse_rep_int'])*8*max_seconds_to_load
load_start_bytes = rx_len_samples*int(1/config['CHIRP']['pulse_rep_int'])*8*load_start_seconds

file_size_bytes = os.path.getsize(rx_samps) - load_start_bytes
if file_size_bytes > max_file_size_bytes:
    print(f"WARNING: File is {file_size_bytes/(2**30):.2f} GB ({file_size_bytes / (rx_len_samples*int(1/config['CHIRP']['pulse_rep_int'])*2):.2f} seconds). Only loading the first {max_seconds_to_load} seconds.")
    file_size_bytes = max_file_size_bytes

rx_sig = np.zeros((file_size_bytes//8,), dtype=np.csingle)
for start_offset in np.arange(0, file_size_bytes, max_chunk_size_samples*8, dtype=np.int64):
    if start_offset + max_chunk_size_samples*8 > file_size_bytes:
        rx_sig[(start_offset//8):] = pr.extractSig(rx_samps, count=(file_size_bytes-start_offset)//4, offset=load_start_bytes+start_offset)
    else:
        rx_sig[(start_offset//8):((start_offset//8)+(max_chunk_size_samples))] = pr.extractSig(rx_samps, count=max_chunk_size_samples*2, offset=load_start_bytes+start_offset)

print(datetime.datetime.now() - t0)

In [ ]:
## Reshape data

n_rxs = len(rx_sig) // rx_len_samples
rx_sig_reshaped = np.transpose(np.reshape(rx_sig, (n_rxs, rx_len_samples), order='C'))
errors_removed = False # Keep track of if the cell removing errors from rx_sig_reshaped has already been run

print(f"len(rx_sig): {len(rx_sig)}")
print(f"n_rxs: {n_rxs}")
print(f"rx_sig shape: {np.shape(rx_sig)}")
print(f"rx_sig_reshaped shape: {np.shape(rx_sig_reshaped)}")

# Extract log information about errors and start timestamp

errors = None
start_timestamp = None

if os.path.exists(log_file):
    errors = {}
    
    log_f = open(log_file, 'r')
    log = log_f.readlines()
    
    for idx, line in enumerate(log):
        if "Receiver error:" in line:
            error_code = re.search("(?:Receiver error: )([\w_]+)", line).groups()[0]
            old_style_regex_serach = re.search("(?:Scheduling chirp )([\d]+)", log[idx-1])
            if old_style_regex_serach is not None:
                chirp_idx = int(re.search("(?:Scheduling chirp )([\d]+)", log[idx-1]).groups()[0])
            else:
                chirp_idx = int(re.search("(?:Chirp )([\d]+)", line).groups()[0])
            errors[chirp_idx] = error_code
            if error_code != "ERROR_CODE_LATE_COMMAND":
                print(f"WARNING: Uncommon error in the log: {error_code} (on chirp {chirp_idx})")
                print(f"Full message: {line}")
        if ("[START]" in line) or ("Scheduling chirp 0 RX" in line):
            start_timestamp = float(re.search("(?:\[)([\d]+\.[\d]+)", line).groups()[0])
else:
    print(f"WARNING: No log file found. This is fine, but checks for error codes will be disabled.")
    print(f"(Looking for a log file in: {log_file})")

# Handle errors

# Choose what to do with chirps with a reported error (usually ERROR_CODE_LATE_COMMAND)
# Options are:
# None - do nothing
# 'zeros' - replace with zeros
# 'remove' - remove them from the rx_sig_reshaped array
error_behavior = None

if errors is None:
    if error_behavior is not None:
        print(f"WARNING: Requested doing something with errors but no log file was loaded. Defaulting to doing nothing.")
elif error_behavior == 'zeros':
    error_idxs = np.array(list(errors.keys()))
    rx_sig_reshaped[:,error_idxs] = 0
elif error_behavior == 'remove':
    if errors_removed:
        print(f"WARNING: Error chirps already removed since rx_sig_reshaped created. Not doing again.")
    else:
        error_idxs = np.array(list(errors.keys()))
        all_idxs = np.arange(rx_sig_reshaped.shape[1])
        keep_idxs = [x for x in all_idxs if x not in error_idxs]

        rx_sig_reshaped = rx_sig_reshaped[:, keep_idxs]
        n_rxs = len(keep_idxs)
        errors_removed = True
    
print(f"n_rxs: {n_rxs}")
print(f"rx_sig_reshaped shape: {np.shape(rx_sig_reshaped)}")
print(f"Extracted start timestamp: {start_timestamp}")

In [ ]:
slow_time = np.linspace(0, config['CHIRP']['pulse_rep_int']*config['CHIRP']['num_presums']*n_rxs, np.shape(rx_sig_reshaped)[1])

for idx in errors:
    slow_time[idx+1:] += config['CHIRP']['pulse_rep_int'] * 3

# TODO: Should play with this more to see if it correctly accounts for drift to to errors

#plt.plot(np.diff(slow_time))

In [ ]:
chirp_idx = 19000
print(slow_time[chirp_idx])
fig, axs = pr.plotChirpVsTime(rx_sig_reshaped[:,chirp_idx], 'Recieved Chirp', sample_rate)
axs[0].set_ylim(-1,1)
axs[1].set_ylim(-1,1)
axs[0].text(0, 1.1, prefix.split("/")[-1] + "\n" + f"chirp_idx = {chirp_idx}", horizontalalignment='left', verticalalignment='center', transform=axs[0].transAxes)
plt.savefig(prefix + "_rx_chirp.png")

In [ ]:
test = rx_sig_reshaped[:, 0:19316:100]
test = test.transpose().flatten()

np.all(test[rx_len_samples*100:rx_len_samples*101] == rx_sig_reshaped[:,100])

In [ ]:
def plot_rx_spectrum(input_rx_sig_reshaped = rx_sig_reshaped, subsample=1, start_time_s=0, duration_s=-1, figsize=None, vmin=None, vmax=None, x_axis_label='Time [s]'):

    if duration_s < 0:
        duration_s = np.max(slow_time) - start_time_s

    start_idx = np.argmin(np.abs(slow_time - start_time_s))
    end_idx = np.argmin(np.abs(slow_time - (start_time_s + duration_s)))
    actual_duration_s = slow_time[end_idx] - slow_time[start_idx]
    
    slow_time_plot = slow_time[np.arange(start_idx, end_idx+1, subsample)]

    rx_sig_cropped = input_rx_sig_reshaped[:, start_idx:end_idx:subsample]
    
    print(start_idx)
    print(end_idx)
    print(np.shape(rx_sig_cropped))
    print(np.shape(slow_time_plot))
    
    rx_sig_tmp = rx_sig_cropped.transpose().flatten()
    
    f, t, Sxx = scipy.signal.spectrogram(rx_sig_tmp, fs = sample_rate, nperseg=rx_len_samples, noverlap=0, mode='magnitude', return_onesided=False)
    
    if figsize is None:
        figsize = (duration_s/10, 5)
            
    fig, ax = plt.subplots(1,1, figsize=figsize, facecolor='white')    

    p = ax.pcolormesh(t, scipy.fft.fftshift(f)/1e6, 10*np.log10(scipy.fft.fftshift(Sxx, axes=0)), shading='gouraud')#, cmap='inferno', vmin=vmin, vmax=vmax)
    clb = fig.colorbar(p, ax=ax)
    clb.set_label('Power Spectral Density [dB/Hz]')
    #ax.set_xlabel('Time [s]')
    ax.set_ylabel('Frequency [MHz]')

    ax.text(0, 1.05, prefix.split("/")[-1] + "\n" + f"subsample = {subsample}", horizontalalignment='left', verticalalignment='center', transform=ax.transAxes)

    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label, clb.ax.yaxis.label] +
                ax.get_xticklabels() + ax.get_yticklabels() + clb.ax.get_yticklabels()): #ax.get_legend().get_texts()
        item.set_fontsize(18)
        item.set_fontfamily('sans-serif')
        
    fig.tight_layout()
    
    return fig, ax

#plot_rx_spectrum(duration_s=500, subsample=100, figsize=(15, 5))

In [ ]:
fig, ax, _ = plot_radargram(n_stack=20, start_time_s=100, duration_s=10, upsampling=1, figsize=(15, 5), vmin=-50, vmax=-0, ylims=(150, 0))

In [ ]:
sample_rate

In [ ]:
print(np.shape(rx_sig_reshaped))

tx_sig_padded = np.zeros((np.shape(rx_sig_reshaped)[0],), dtype=np.complex64)
tx_sig_padded[0:len(tx_sig)] = tx_sig
#tx_sig_padded[3:len(tx_sig)+3] += 0.5*tx_sig

# fig, ax = plt.subplots()
# ax.plot(np.real(tx_sig))
# ax.plot(np.real(tx_sig_padded))
# ax.plot(np.real(rx_sig_reshaped[:,0]))

tx_sig_padded

tx_sig_duplicated = np.transpose(np.tile(np.transpose(tx_sig_padded), (2000,1)))
print(np.shape(tx_sig_duplicated))

plot_rx_spectrum(input_rx_sig_reshaped = tx_sig_duplicated, duration_s=1, subsample=1, figsize=(15, 5))

In [ ]:
def plot_radargram(n_stack=15, start_time_s=0, duration_s=-1, upsampling=1, figsize=None, vmin=-70, vmax=-40, ylims=(65, 15), surface_dist=None, x_axis=None, x_axis_label='Time [s]'):

    if duration_s < 0:
        duration_s = np.max(slow_time) - start_time_s

    start_idx = np.argmin(np.abs(slow_time - start_time_s))
    end_idx = np.argmin(np.abs(slow_time - (start_time_s + duration_s)))
    actual_duration_s = slow_time[end_idx] - slow_time[start_idx]

    rx_sig_cropped = rx_sig_reshaped[:, start_idx:end_idx]

    corr_sig = tx_sig_blackman_window

    if upsampling > 1:
        corr_sig = scipy.signal.resample_poly(corr_sig, upsampling, 1)

    xcorr_results = np.zeros((((rx_len_samples*upsampling)-len(corr_sig))+1, np.shape(rx_sig_cropped)[1]//n_stack), dtype=np.csingle)

    if surface_dist is not None:
        surface_dist_plot = surface_dist[np.arange(start_idx, end_idx-n_stack, n_stack)]
    
    slow_time_plot = slow_time[np.arange(start_idx, end_idx-n_stack+1, n_stack)]
    if x_axis is None:
        x_axis_plot = slow_time_plot
    else:
        x_axis_plot = x_axis[np.arange(start_idx, end_idx-n_stack+1, n_stack)]
    
    distance_to_reflector = np.linspace(0, np.shape(xcorr_results)[0]/(sample_rate*upsampling), np.shape(xcorr_results)[0]) * sig_speed / 2
    distance_to_reflector = distance_to_reflector - distance_to_reflector[zero_sample_idx*upsampling]

    for res_idx in range(np.shape(xcorr_results)[1]):
        stacked = np.mean(rx_sig_cropped[:,res_idx*n_stack:(res_idx+1)*n_stack], axis=1)
        if upsampling > 1:
            stacked = scipy.signal.resample_poly(stacked, upsampling, 1)

        if surface_dist is not None:
            surf_idx_offset = np.argmin(np.abs(distance_to_reflector - surface_dist_plot[res_idx])) - np.argmin(np.abs(distance_to_reflector))
            xcorr_results[:-surf_idx_offset, res_idx] = sp.correlate(stacked, corr_sig, mode='valid', method='auto')[surf_idx_offset:] / np.sum(np.abs(corr_sig)**2)
        else:
            xcorr_results[:, res_idx] = sp.correlate(stacked, corr_sig, mode='valid', method='auto') / np.sum(np.abs(corr_sig)**2)
        
    if figsize is None:
        figsize = (duration_s/10, 5)
            
    fig, ax = plt.subplots(1,1, figsize=figsize, facecolor='white')

    

    return_power = 20*np.log10(np.abs(xcorr_results))
    p = ax.pcolormesh(x_axis_plot, distance_to_reflector, return_power, shading='auto', cmap='inferno', vmin=vmin, vmax=vmax)
    clb = fig.colorbar(p, ax=ax)
    clb.set_label('Power [dB]')
    ax.set_xlabel(x_axis_label)
    ax.set_ylabel('Distance to reflector [m]')

    ax.set_ylim(ylims[0], ylims[1])
    #ax.set_xlim(start_time_s,start_time_s+duration_s)
    ax.set_xlim(np.min(x_axis_plot), np.max(x_axis_plot))

    ax.text(0, 1.05, prefix.split("/")[-1] + "\n" + f"n_stack = {n_stack}", horizontalalignment='left', verticalalignment='center', transform=ax.transAxes)

    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label, clb.ax.yaxis.label] +
                ax.get_xticklabels() + ax.get_yticklabels() + clb.ax.get_yticklabels()): #ax.get_legend().get_texts()
        item.set_fontsize(18)
        item.set_fontfamily('sans-serif')
        
    fig.tight_layout()

    return fig, ax, {'slow_time': slow_time_plot, 'distance_to_reflector': distance_to_reflector, 'return_power': return_power}

In [ ]:
fig, ax, _ = plot_radargram(n_stack=20, start_time_s=0, duration_s=100, upsampling=1, figsize=(15, 5), vmin=-90, vmax=-40, ylims=(300, 10))

In [ ]:
#fig, ax, _ = plot_radargram(n_stack=20, start_time_s=0, duration_s=-1, upsampling=1, figsize=(15, 5), vmin=-90, vmax=-35, ylims=(150, 5))

## Single Timestep

In [ ]:
# Must enable widgets (see imports block) for this to work

def plot_timestep(start_chirp_idx, n_stack):
    
    stacked = np.mean(rx_sig_reshaped[:,start_chirp_idx:start_chirp_idx+n_stack], axis=1)
    
    xcorr_res_tmp = sp.correlate(stacked, tx_sig_blackman_window, mode='valid', method='direct')
    
    slow_time = np.linspace(0,config['CHIRP']['pulse_rep_int']*config['CHIRP']['num_presums']*n_rxs, np.shape(rx_sig_reshaped)[1])
    
    distance_to_reflector = np.linspace(0, np.shape(xcorr_res_tmp)[0]/sample_rate, np.shape(xcorr_res_tmp)[0]) * sig_speed / 2
    distance_to_reflector = distance_to_reflector - distance_to_reflector[zero_sample_idx]
    
    fig, ax = plt.subplots(figsize=(10,5), facecolor='white')
    line = ax.plot(distance_to_reflector, 20*np.log10(np.abs(xcorr_res_tmp)), label="Data")
    mplcursors.cursor(line)
    #ax.set_ylim(0,100)
    ax.set_ylim(-40,80)
    ax.set_ylim(-20,40)
    #ax.set_ylim(-20,80)
    ax.set_xlim(-20, 120)
    #ax.set_xlim(-20, 700)
    ax.set_xlim(-200,600)
    ax.set_xlim(-10,250)
    #ax.set_xlim(-500,500)
    ax.set_xlabel('Distance [m]')
    ax.set_ylabel('Power [dB]')
    ax.set_title(f't = {slow_time[start_chirp_idx]:.2f} s')
    ax.grid()
    
    direct_path_time_offset = distance_to_reflector[np.argmax(np.abs(xcorr_res_tmp))]
    
    #ax.plot(distance_to_reflector_autocorr + direct_path_time_offset, 20*np.log10(np.abs(tx_sig_rx_envelope_corr)), label="TX sig w/ RX envelope scaling corr", alpha=0.2)
    #ax.plot(-1*distance_to_reflector + 2*direct_path_time_offset, 20*np.log10(np.abs(xcorr_res_tmp)), label="Data (distance flipped)", alpha=0.2)
    
    # Theoretical TX sig autocorr
    tx_sig_autocorr = sp.correlate(tx_sig, tx_sig, mode='same', method='auto')
    tx_sig_autocorr_db = 20*np.log10(np.abs(tx_sig_autocorr))

    distance_to_reflector_autocorr = np.linspace(0, len(tx_sig_autocorr)/sample_rate, len(tx_sig_autocorr)) * sig_speed / 2
    distance_to_reflector_autocorr = distance_to_reflector_autocorr - distance_to_reflector_autocorr[len(tx_sig_autocorr)//2]

    #ax.plot(distance_to_reflector_autocorr, tx_sig_autocorr_db - np.max(tx_sig_autocorr_db) + np.max(20*np.log10(np.abs(xcorr_res_tmp))), label="Theoretical TX sig autocorr")
    
    # rx_sig_autocorr_peak_idx = np.argmax(np.abs(rx_sig_autocorr))
    # xcorr_peak_idx = np.argmax(np.abs(xcorr_res_tmp))
    # print(xcorr_peak_idx)
    # rx_sig_autocorr_aligned = rx_sig_autocorr[rx_sig_autocorr_peak_idx - xcorr_peak_idx : rx_sig_autocorr_peak_idx + (len(xcorr_res_tmp) - xcorr_peak_idx)]
    # rx_sig_autocorr_aligned = rx_sig_autocorr_aligned  * (np.max(np.abs(xcorr_res_tmp)) / np.max(np.abs(rx_sig_autocorr_aligned))) # scale peak

    #ax.plot(distance_to_reflector, 20*np.log10(rx_sig_autocorr_aligned))
    
    #ax.plot(distance_to_reflector, 20*np.log10(np.maximum(np.abs(xcorr_res_tmp) - 0.95*np.abs(rx_sig_autocorr_aligned), 0.1)))
    direct_path_sample_idx = np.argmax(np.abs(xcorr_res_tmp))
    xcorr_subtracted = (np.abs(xcorr_res_tmp))
    xcorr_subtracted[direct_path_sample_idx:2*direct_path_sample_idx] -= (np.abs(xcorr_res_tmp[direct_path_sample_idx:0:-1]))

    #np.abs(xcorr_res_tmp[:-(direct_path_sample_offset)]) - 
    #ax.plot(distance_to_reflector, 20*np.log10(np.abs(xcorr_subtracted)))
    
    #ax.plot(distance_to_reflector_autocorr + direct_path_time_offset, 20*np.log10(np.abs(rx_sig_autocorr_mean)))
    
    #ax.legend()
    
    ax.text(0, 1.05, prefix.split("/")[-1] + "\n" + f"n_stack = {n_stack}, start_chirp_idx = {start_chirp_idx}", horizontalalignment='left', verticalalignment='center', transform=ax.transAxes)
    
    return fig, ax
    
widgets.interact(plot_timestep,
                 start_chirp_idx=widgets.IntSlider(min=0, max=np.shape(rx_sig_reshaped)[1]-1, step=1, value=0),
                 n_stack=widgets.IntSlider(min=1, max=100, step=1, value=50))

### SNR Estimation

In [ ]:
reference_range = [40, 60]
surface_range = [20, 30]

verbose = False

n_stacks = np.sort(list(set(np.logspace(np.log10(1), np.log10(500), 20).astype(int)))) #np.arange(1, 150, 5)
#n_stacks = np.array([1, 5, 10, 30, 100])
sig, noise = np.zeros_like(n_stacks, dtype=float), np.zeros_like(n_stacks, dtype=float)

for idx, n_stack in enumerate(n_stacks):
    fig, _, radargram_data = plot_radargram(n_stack=n_stack, start_time_s=0, duration_s=-1, upsampling=1, figsize=(15, 5), vmin=-100, vmax=-35, ylims=(60, -10))
    #fig, _, radargram_data = phase_align_test(n_stack=n_stack, start_time_s=0, duration_s=-1, upsampling=1, figsize=(15, 5), vmin=-100, vmax=-35, ylims=(60, -10))
    
    fig.savefig(f"figures/tmp_gif/{prefix.split('/')[-1]}_nstack_{n_stack:03d}.png")
    
#     fig, ax = plt.subplots(figsize=(15,5), facecolor='white')
#     ax.plot(radargram_data['distance_to_reflector'], radargram_data['return_power'][:, 0])
#     ax.set_xlim(-10,60)
#     ax.set_ylim(-100,-30)
#     ax.grid()
#     ax.set_xlabel('Distance to reflector [m]')
#     ax.set_ylabel('Return power [dB]')
#     ax.set_title(f"[{prefix.split('/')[-1]}] n_stack = {n_stack}")
    
#     fig.savefig(f"figures/tmp_gif/{prefix.split('/')[-1]}_1d_nstack_{n_stack:03d}.png")
    
    surface_idxs_mask = (radargram_data['distance_to_reflector'] <= surface_range[1]) & (radargram_data['distance_to_reflector'] >= surface_range[0])
    noise_reference_idxs_mask = (radargram_data['distance_to_reflector'] <= reference_range[1]) & (radargram_data['distance_to_reflector'] >= reference_range[0])

    if verbose:
        print(f"Using {reference_range[0]} m to {reference_range[1]} m as the noise floor. ({np.sum(noise_reference_idxs_mask)} range bins)")
        print(f"Searching for surface peak from {surface_range[0]} m to {surface_range[1]} m as the noise floor. ({np.sum(surface_idxs_mask)} range bins)")
        print(f"Taking median values over {len(radargram_data['slow_time'])} along-track samples")

    surface_peaks_db = np.zeros_like(radargram_data['slow_time'])
    noise_means_db = np.zeros_like(radargram_data['slow_time'])

    for time_idx in range(len(radargram_data['slow_time'])):
        pwr = radargram_data['return_power'][:, time_idx]
        surface_peaks_db[time_idx] = np.max(pwr[surface_idxs_mask])
        noise_reference_values_db = pwr[noise_reference_idxs_mask]
        noise_reference_values_linear = 10**(noise_reference_values_db / 20)
        noise_means_db[time_idx] = 20 * np.log10(np.mean(noise_reference_values_linear))
        
    sig[idx] = np.median(surface_peaks_db)
    noise[idx] = np.median(noise_means_db)
    
    print(f"[n_stack = {n_stack}] \t", end="")
    print(f"Median surface peak: {np.median(surface_peaks_db):.2f} dB \tMedian noise: {np.median(noise_means_db):.2f} dB", end="")
    print(f" \tSNR: {np.median(surface_peaks_db) - np.median(noise_means_db):.2f} dB")    

In [ ]:
fig, axs = plt.subplots(2,1, figsize=(15, 10), facecolor='white')

axs[0].scatter(n_stacks, sig, label='Signal [dB]')
axs[0].scatter(n_stacks, noise, label='Noise [dB]')
axs[0].legend()
axs[0].set_ylabel('Power [dB]')
axs[0].set_title('Signal and noise estimated power [dB]')
axs[0].grid()
axs[0].set_xlabel('n_stack')

snrs = sig - noise
snr_lin = 10**(snrs / 20)

assert n_stacks[0] == 1

axs[1].scatter(n_stacks, snr_lin, c='red', label='Estimted from data')
axs[1].plot(n_stacks, n_stacks*snr_lin[0], label='Theoretical coherent summation')
axs[1].plot(n_stacks, np.sqrt(n_stacks)*snr_lin[0], label='Theoretical incoherent summation')
axs[1].set_ylabel('SNR [dimensionless, linear scale]')
axs[1].set_title('Linear scale SNR')
axs[1].legend()
axs[1].grid()
axs[1].set_xlabel('n_stack')
axs[1].set_ylim(0, 50)

plt.show()

### Phase Alignment

In [ ]:
def phase_align_test(n_stack=15, start_time_s=0, duration_s=-1, upsampling=1, figsize=None, vmin=-70, vmax=-40, ylims=(65, 15), surface_dist=None, x_axis=None, x_axis_label='Time [s]', phase_align=True):

    if duration_s < 0:
        duration_s = np.max(slow_time) - start_time_s

    start_idx = np.argmin(np.abs(slow_time - start_time_s))
    end_idx = np.argmin(np.abs(slow_time - (start_time_s + duration_s)))
    actual_duration_s = slow_time[end_idx] - slow_time[start_idx]

    rx_sig_cropped = rx_sig_reshaped[:, start_idx:end_idx]

    corr_sig = tx_sig_blackman_window

    if upsampling > 1:
        corr_sig = scipy.signal.resample_poly(corr_sig, upsampling, 1)

    xcorr_results = np.zeros((((rx_len_samples*upsampling)-len(corr_sig))+1, np.shape(rx_sig_cropped)[1]), dtype=np.csingle)
    xcorr_results_stacked = np.zeros((((rx_len_samples*upsampling)-len(corr_sig))+1, np.shape(rx_sig_cropped)[1]//n_stack), dtype=np.csingle)

    if surface_dist is not None:
        surface_dist_plot = surface_dist[np.arange(start_idx, end_idx-n_stack, n_stack)]
    
    slow_time_plot = slow_time[np.arange(start_idx, end_idx-n_stack+1, n_stack)]
    if x_axis is None:
        x_axis_plot = slow_time_plot
    else:
        x_axis_plot = x_axis[np.arange(start_idx, end_idx-n_stack+1, n_stack)]
    
    distance_to_reflector = np.linspace(0, np.shape(xcorr_results)[0]/(sample_rate*upsampling), np.shape(xcorr_results)[0]) * sig_speed / 2
    distance_to_reflector = distance_to_reflector - distance_to_reflector[zero_sample_idx*upsampling]

    for res_idx in range(np.shape(rx_sig_cropped)[1]):
        #stacked = np.mean(rx_sig_cropped[:,res_idx*n_stack:(res_idx+1)*n_stack], axis=1)
        trace = rx_sig_cropped[:,res_idx]
        
        if upsampling > 1:
            trace = scipy.signal.resample_poly(trace, upsampling, 1)

        xcorr_results[:, res_idx] = sp.correlate(trace, corr_sig, mode='valid', method='auto') / np.sum(np.abs(corr_sig)**2)
        
    if phase_align:
        peaks_direct = np.argmax(np.abs(xcorr_results), axis=0)
        direct = [xcorr_results[x,idx] for idx, x in enumerate(peaks_direct)]
        xcorr_aligned = np.array([xcorr_results[:,idx] * np.exp(-1j * np.angle(direct[idx])) for idx in range(np.shape(rx_sig_cropped)[1])])
        xcorr_results = np.transpose(xcorr_aligned)
        
    for res_idx in range(np.shape(xcorr_results_stacked)[1]):
        xcorr_results_stacked[:, res_idx] = np.mean(xcorr_results[:,res_idx*n_stack:(res_idx+1)*n_stack], axis=1)
    
    
    # TODO: Support surf_dist again -- after all other processing
    
    if figsize is None:
        figsize = (duration_s/10, 5)
            
    fig, ax = plt.subplots(1,1, figsize=figsize, facecolor='white')

    

    return_power = 20*np.log10(np.abs(xcorr_results_stacked))
    p = ax.pcolormesh(x_axis_plot, distance_to_reflector, return_power, shading='auto', cmap='inferno', vmin=vmin, vmax=vmax)
    clb = fig.colorbar(p, ax=ax)
    clb.set_label('Power [dB]')
    ax.set_xlabel(x_axis_label)
    ax.set_ylabel('Distance to reflector [m]')

    ax.set_ylim(ylims[0], ylims[1])
    #ax.set_xlim(start_time_s,start_time_s+duration_s)
    ax.set_xlim(np.min(x_axis_plot), np.max(x_axis_plot))

    ax.text(0, 1.05, prefix.split("/")[-1] + "\n" + f"n_stack = {n_stack}", horizontalalignment='left', verticalalignment='center', transform=ax.transAxes)

    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label, clb.ax.yaxis.label] +
                ax.get_xticklabels() + ax.get_yticklabels() + clb.ax.get_yticklabels()): #ax.get_legend().get_texts()
        item.set_fontsize(18)
        item.set_fontfamily('sans-serif')
        
    fig.tight_layout()

    return fig, ax, {'slow_time': slow_time_plot, 'distance_to_reflector': distance_to_reflector, 'return_power': return_power,
                     'peaks_direct': peaks_direct, 'xcorr_results_stacked': xcorr_results_stacked}

In [ ]:
fig, ax, radargram_data = phase_align_test(n_stack=10, start_time_s=250, duration_s=10, upsampling=1, figsize=(15, 5), vmin=-70, vmax=-10, ylims=(80, 15))


In [ ]:
radargram_data['xcorr_results_stacked'].shape

In [ ]:
fig, ax = plt.subplots()
ax.plot(radargram_data['distance_to_reflector'], np.abs(radargram_data['xcorr_results_stacked'][:,100]))

In [ ]:
np.argmax(np.abs(radargram_data['xcorr_results_stacked']), axis=0)

In [ ]:
radargram_data['peaks_direct'].shape

In [ ]:
np.shape(radargram_data['xcorr_results'])

In [ ]:
direct = [radargram_data['xcorr_results'][x,idx] for idx, x in enumerate(radargram_data['peaks_direct'])]
xcorr_aligned = np.array([radargram_data['xcorr_results'][:,idx] * np.exp(-1j * np.angle(direct[idx])) for idx in range(len(radargram_data['slow_time']))])
xcorr_aligned = np.transpose(xcorr_aligned)

In [ ]:
peak_idxs = np.argmax(np.abs(radargram_data['xcorr_results_stacked']), axis=0)
direct = [radargram_data['xcorr_results_stacked'][x,idx] for idx, x in enumerate(peak_idxs)]
fig, axs = plt.subplots(3,1, figsize=(15, 10), facecolor='white')

axs[0].plot(radargram_data['slow_time'], np.abs(direct))
axs[0].set_ylabel('xcorr peak magnitude (linear scale)')

axs[1].plot(radargram_data['slow_time'], np.angle(direct))
#axs[1].set_ylim(-1.1, -0.9)
#axs[1].set_ylim(-0.1, 0.1)
axs[1].set_ylabel('xcorr peak phase [rad]')

axs[2].plot(radargram_data['slow_time'], peak_idxs)
axs[2].set_ylabel('index of xcorr peak')

### Back to normal stuff

In [ ]:
if px4_logs is not None:
    # Get UTC time from GPS
    df = pd.read_csv(px4_logs + "sensor_gps_0.csv")
    px4_timestamp_offset_to_utc = (df['time_utc_usec'].at[0] / 1e6) - (df['timestamp'].at[0] / 1e6)

    slow_time_unix = slow_time + start_timestamp

    def interp_log_to_radar(df, key, interpolation_kind='linear'):
        ts = df['timestamp']*1e-6 + px4_timestamp_offset_to_utc

        interp_fn = scipy.interpolate.interp1d(ts, df[key], kind=interpolation_kind, bounds_error=False)
        return interp_fn(slow_time_unix)

    px4_logdata = {'time_s': slow_time}

    # Get takeoff/landing times
    df = pd.read_csv(px4_logs + "vehicle_land_detected_0.csv")
    landed = df.loc[df['landed'] == 0]
    takeoff, landing = landed['timestamp'].min(), landed['timestamp'].max()
    takeoff_radar_timestamp, landing_radar_timestamp = [t*1e-6 + px4_timestamp_offset_to_utc - start_timestamp for t in [takeoff, landing]]
    flight_duration = landing_radar_timestamp - takeoff_radar_timestamp
    print((takeoff_radar_timestamp, landing_radar_timestamp))

    # Distance traveled and local positions
    df = pd.read_csv(px4_logs + "estimator_local_position_0.csv")
    local_x, local_y, local_z = df['x'], df['y'], df['z']
    dist_xy_plane = np.cumsum(np.sqrt(np.diff(local_x)**2 + np.diff(local_y)**2))
    df['distance_traveled'] = np.concatenate([[0], dist_xy_plane])
    px4_logdata['distance_traveled'] = interp_log_to_radar(df, 'distance_traveled')
    for k in ['x', 'y', 'z']:
        px4_logdata[k] = interp_log_to_radar(df, k)

    # Target bearing
    df = pd.read_csv(px4_logs + "position_controller_status_0.csv")
    for k in ['target_bearing', 'nav_roll']:
        px4_logdata[k] = interp_log_to_radar(df, k)
    
    # Mission item index
    df = pd.read_csv(px4_logs + "navigator_mission_item_0.csv")
    px4_logdata['mission_item'] = interp_log_to_radar(df, 'sequence_current', interpolation_kind='previous')

    # Load distance sensor data
    df_distance_sensor = pd.read_csv(px4_logs + "distance_sensor_0.csv")
    df_distance_sensor['dist_filt'] = df_distance_sensor['current_distance']
    df_distance_sensor['dist_filt'].iloc[df_distance_sensor['dist_filt'] > 120] = None
    px4_logdata['dist'] = interp_log_to_radar(df_distance_sensor, 'dist_filt')

    df_position = pd.read_csv(px4_logs + "estimator_global_position_0.csv")
    df_position['alt_rel'] = df_position['alt'] - df_position['alt'].min()
    for k in ['lat', 'lon', 'alt_rel']:
        px4_logdata[k] = interp_log_to_radar(df_position, k)
        
    # Vehicle attitude
    df_attitude = pd.read_csv(px4_logs + "vehicle_attitude_0.csv")
    att = R.from_quat(df_attitude[['q[0]', 'q[1]', 'q[2]', 'q[3]']])
    att_ypr = att.as_euler('xyz', degrees=True)
    df_attitude['yaw'] = att_ypr[:,0]
    df_attitude['pitch'] = att_ypr[:,1]
    df_attitude['roll'] = att_ypr[:,2]
    for k in ['yaw', 'pitch', 'roll']:
        px4_logdata[k] = interp_log_to_radar(df_attitude, k)

    df_px4 = pd.DataFrame(px4_logdata, index=slow_time)

    df_px4

In [ ]:
if px4_logs is not None:
    fig, ax, plot_data = plot_radargram(n_stack=15, start_time_s=takeoff_radar_timestamp, duration_s=flight_duration, upsampling=3, figsize=(15, 5), vmin=-70, vmax=-40, ylims=(80, 15), x_axis=px4_logdata['distance_traveled']/1000, x_axis_label='Distance [km]')
    ax.scatter(px4_logdata['distance_traveled']/1000, df_px4['dist'] / np.sqrt(3.17), c='white', s=1)
    #ax.plot(px4_logdata['distance_traveled']/1000, d_interp, c='white')
    ax.plot(px4_logdata['distance_traveled']/1000, df_px4['alt_rel'] / np.sqrt(3.17))

In [ ]:
fig, ax, plot_data = plot_radargram(n_stack=30, start_time_s=takeoff_radar_timestamp, duration_s=flight_duration, upsampling=1, figsize=(15, 5), vmin=-80, vmax=-40, ylims=(300, 10), x_axis=px4_logdata['distance_traveled']/1000, x_axis_label='Distance [km]')
#fig.savefig(prefix + "_radargram.png")

In [ ]:
fig, ax, plot_data = plot_radargram(n_stack=40, start_time_s=takeoff_radar_timestamp+60, duration_s=flight_duration-120, upsampling=1, figsize=(15, 5), vmin=-90, vmax=-10, ylims=(250, 10))#, x_axis=px4_logdata['distance_traveled']/1000, x_axis_label='Distance [km]')

In [ ]:
px4_logdata_df = pd.DataFrame(px4_logdata)

mission_idx_diff = np.diff(px4_logdata_df['mission_item'])
mission_idx_diff[np.isnan(mission_idx_diff)] = 0

mission_segments = np.split(np.array(px4_logdata_df.index), np.squeeze(np.argwhere(mission_idx_diff != 0)))

for segment_idx, segment_df_idxs in enumerate(mission_segments):
    px4_logdata_segment_df = px4_logdata_df.iloc[segment_df_idxs, :].copy()
    segment_distance = px4_logdata_segment_df['distance_traveled'].max() - px4_logdata_segment_df['distance_traveled'].min()
    if segment_distance < 300:
        continue # skip any short segments

    # Optional -- remove first n seconds of the segment -- usually a turn
    px4_logdata_segment_df = px4_logdata_segment_df[px4_logdata_segment_df['time_s'] - px4_logdata_segment_df['time_s'].min() > 5]
    
    start_time = px4_logdata_segment_df['time_s'].min()
    duration = px4_logdata_segment_df['time_s'].max() - start_time - 1e-6

    # Context plot
    fig, ax = plt.subplots(figsize=(5,5))
    ax.plot(px4_logdata_df['x'], px4_logdata_df['y'])
    ax.plot(px4_logdata_segment_df['x'], px4_logdata_segment_df['y'], c='red')
    ax.set_aspect('equal')
    ax.scatter([np.array(px4_logdata_segment_df['x'])[0]], [np.array(px4_logdata_segment_df['y'])[0]], c='green', label='Start')
    ax.legend()

    print(f"start: {start_time} \tduration: {duration}")

    # Radargram
    fig, ax, _ = plot_radargram(n_stack=40, start_time_s=start_time, duration_s=duration, upsampling=10, figsize=(10*segment_distance/300, 6), vmin=-80, vmax=-40, ylims=(200, 10), x_axis=px4_logdata_df['distance_traveled']/1000, x_axis_label='Distance [km]')

In [ ]:
takeoff_radar_timestamp, flight_duration

In [ ]:
fig, ax, plot_data = plot_radargram(n_stack=30, start_time_s=141, duration_s=150, upsampling=5, figsize=(20, 5), vmin=-80, vmax=-35, ylims=(150, 10), x_axis=px4_logdata['distance_traveled']/1000, x_axis_label='Flight distance from takeoff [km]')

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
ax.plot(px4_logdata['distance_traveled']/1000, px4_logdata['nav_roll'])
ax.set_xlim(1.5, 4)

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.scatter(df_px4['time_s'], df_px4['dist'] / np.sqrt(3.17), c='blue', s=1)

d = df_px4['dist'].to_numpy() / np.sqrt(3.17)

laser_delta = np.diff(d) / np.diff(df_px4['time_s'].to_numpy())
laser_delta = np.concatenate([laser_delta, [0]])
print(np.shape(laser_delta))

d[np.isnan(laser_delta)] = np.nan
d[np.abs(laser_delta) > 2] = np.nan

d[d > 50] = np.nan
d[d < 20] = np.nan

interp = scipy.interpolate.interp1d((df_px4['time_s'].to_numpy())[~np.isnan(d)], d[~np.isnan(d)], kind='cubic', bounds_error=False)
d_interp = interp(df_px4['time_s'])

ax.scatter(df_px4['time_s'], d, c='red', s=1)
ax.plot(df_px4['time_s'], d_interp, c='orange')

#ax.scatter((df_px4['time_s'].to_numpy())[:-1], np.diff(d) / np.diff(df_px4['time_s'].to_numpy()))


In [ ]:
fig, ax, plot_data = plot_radargram(n_stack=15, start_time_s=350, duration_s=100, upsampling=10, figsize=(15, 5), vmin=-65, vmax=-45, ylims=(60, 15))
fig, ax, plot_data = plot_radargram(n_stack=15, start_time_s=350, duration_s=100, upsampling=10, figsize=(15, 5), vmin=-65, vmax=-45, ylims=(35, -10), surface_dist=d_interp)

In [ ]:
np.argmin(np.abs(plot_data['distance_to_reflector'])), np.argmin(np.abs(plot_data['distance_to_reflector'] - 60))

In [ ]:
s = ret_pwr[318:398,1000]
peaks, _ = scipy.signal.find_peaks(s, prominence=(10, None), height=-60)
plt.plot(s)
plt.scatter(peaks, s[peaks])

In [ ]:
ret_pwr = plot_data['return_power']
peak_idxs = np.zeros((np.shape(ret_pwr)[1],), dtype=int) * np.nan

peak_dists = np.zeros(peak_idxs.shape) * np.nan

for idx in range(ret_pwr.shape[1]):
    peaks, _ = scipy.signal.find_peaks(ret_pwr[318:398, idx], prominence=(10, None), height=-60)
    if len(peaks) > 0:
        peak_idxs[idx] = peaks[0] + 318
        peak_dists[idx] = plot_data['distance_to_reflector'][peaks[0] + 318]



plt.plot(peak_dists)

In [ ]:
peak_dists_filt = scipy.signal.medfilt(peak_dists, 11)
plt.plot(peak_dists_filt)

In [ ]:
fig, ax, plot_data = plot_radargram(n_stack=30, start_time_s=100, duration_s=700, upsampling=5, figsize=(20, 5), vmin=-80, vmax=-40, ylims=(130, 10), x_axis=px4_logdata['distance_traveled']/1000, x_axis_label='Flight distance from takeoff [km]')

In [ ]:
# Dump interpolated data to a pickle file

px4_interp = {'time_s': plot_data['slow_time']}

for k in df_px4:
    interp = scipy.interpolate.interp1d(df_px4['time_s'], df_px4[k], kind='linear', bounds_error=False)
    px4_interp[k] = interp(plot_data['slow_time'])

df_px4_interp = pd.DataFrame(px4_interp)

start_idx, end_idx = np.argmin(np.abs(plot_data['distance_to_reflector'] - 10)), np.argmin(np.abs(plot_data['distance_to_reflector'] - 200))

data = {
    'return_power': plot_data['return_power'][start_idx:end_idx, :], # (2241 by 2666) where 2241 is the number of range bins and 2666 is the number of timesteps
    'slow_time': plot_data['slow_time'], # 2666 -- relative timestamps in seconds
    'distance_to_reflector': plot_data['distance_to_reflector'][start_idx:end_idx], # 2441 -- one-way distance to a reflector assuming propagation velocity in ice (speed of light / sqrt(3.17))
    'lat': df_px4_interp['lat'].to_numpy(), # latitude
    'lon': df_px4_interp['lon'].to_numpy(), # longitude
    'alt_rel': df_px4_interp['alt_rel'].to_numpy(),
    'dist': df_px4_interp['dist'].to_numpy()
}

with open('20230315_flight3.pickle', 'wb') as f:
   pickle.dump(data, f)

# Code to read:
# import pickle
# import numpy as np

# with open('20220901_flight2_full.pickle', 'rb') as f:
#     data = pickle.load(f)

# for k in data:
#     print(f"[{k}] shape: {np.shape(data[k])}")
